In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('survival.csv')

In [3]:
df.rename(columns={'Vibration - X':'VibrationX','Vibration  Y':'VibrationY','Vibration  Z':'VibrationZ'},inplace=True)

In [4]:
df1 = df.drop(['Machine ID','Responsible Failure Cause','Broken'],axis=1)

In [5]:
df1_cat = df1[['Machine Name','Failure Model']]

In [6]:
df1['Failure Model'].value_counts()

Healthy State       478
Vibration Model     421
Thermal Model       149
Electrical Model    124
Lifetime Model       83
Name: Failure Model, dtype: int64

In [7]:
LE = LabelEncoder()

In [8]:
df1_cat['Machine_Name'] = LE.fit_transform(df1_cat['Machine Name'])

In [9]:
df1_cat['Failure_Model'] = LE.fit_transform(df1['Failure Model'])

In [10]:
dummy_var = df1_cat[['Machine_Name','Failure_Model']]

In [11]:
df1_num = df1.select_dtypes(include=np.number)

In [12]:
X = pd.concat([df1_num,dummy_var],axis=1)

In [13]:
df1_target = X['Failure_Model']
df1_feature = X.drop('Failure_Model',axis=1)

In [14]:
df1_target.value_counts()

1    478
4    421
3    149
0    124
2     83
Name: Failure_Model, dtype: int64

In [15]:
X_train,X_test,y_train,y_test = train_test_split(df1_feature,df1_target,test_size=0.3,random_state=10)

In [16]:
decision_tree_classification = DecisionTreeClassifier(criterion='entropy',random_state=10)

In [17]:
decision_tree = decision_tree_classification.fit(X_train,y_train)

In [18]:
y_pred_decision_tree = decision_tree.predict(X_test)

In [23]:
y_pred_decision_tree[0:5]

array([3, 2, 4, 1, 4])

In [20]:
prediction1 = pd.DataFrame(y_pred_decision_tree, columns=['predictions'])
prediction1['predictions'] = prediction1['predictions'].astype(int)
prediction1["predictions"] = prediction1['predictions'].map({0:'Electrical Model', 1:'Healthy State',2:'Lifetime Model',3:'Thermal Model',4:'Vibration Model'})

In [26]:
prediction1[0:5]

,predictions
0,Thermal Model
1,Lifetime Model
2,Vibration Model
3,Healthy State
4,Vibration Model


In [27]:
a = list(prediction1['predictions'])

In [28]:
import json

In [29]:
b = json.dumps(a)

In [50]:
 #get prediction
input_cols = [[81,30, 230,1.5,28,1.004,1.143,9.125,1]]
prediction = decision_tree.predict(input_cols)
prediction1 = pd.DataFrame(prediction, columns=['predictions'])
prediction1['predictions'] = prediction1['predictions'].astype(int)
prediction1["predictions"] = prediction1['predictions'].map({0:'Electrical Model', 1:'Healthy State',2:'Lifetime Model',3:'Thermal Model',4:'Vibration Model'})
output1 = list(prediction1['predictions'])
output = json.dumps(output1)

In [51]:
output

'["Lifetime Model"]'

In [ ]:
X_test.head()

In [ ]:
shakjkjdsj

In [ ]:
import pickle

In [ ]:
with open('Multiclass_DT.pkl', 'wb') as files:
    pickle.dump(decision_tree, files)

In [ ]:
with open('Multiclass_DT.pkl' , 'rb') as f:
    Bdt = pickle.load(f)

In [ ]:
from flask import Flask, render_template, request
app = Flask(__name__)
Bdt = pickle.load(open("Multiclass_DT.pkl",'rb'))
@app.route("/")
def home():
    return render_template('Multiclass_machine.html')
@app.route("/predict", methods=['GET','POST'])
def predict():
    if request.method == 'POST':
        #access the data from form

        Lifetime = int(request.form["Lifetime"])
        Temperature = float(request.form["Temperature"])
        Voltage = float(request.form["Voltage"])
        Current = float(request.form["Current"])
        Humidity = float(request.form["Humidity"])
        VibrationX = float(request.form["VibrationX"])
        VibrationY = float(request.form["VibrationY"])
        VibrationZ = float(request.form["VibrationZ"])
        Machine_Name = int(request.form['Machine_Name'])
        
        
        
        #get prediction
        input_cols = [[Lifetime,Temperature, Voltage, Current, Humidity,VibrationX, VibrationY, VibrationZ,Machine_Name]]
        prediction = Bdt.predict(input_cols)
        output = round(prediction[0], 2)
        return render_template("Multiclass_machine.html", prediction_text='Your predicted failure reason of machine is  {}'.format(output))
if __name__ == "__main__":
    app.run()